In [ ]:
#!/usr/bin/env python3 
#！！final one that I use to generate the input files for 5m p s
# Hurricane wind + wind-stress for MOM6 data_table ("bilinear")
# - Output: Taux/Tauy/u10/v10(Time, Lat, Lon) with 1-D Lat/Lon in degrees
# - Time axis includes a +1 record tail to avoid "after range" errors

import os
import numpy as np
from netCDF4 import Dataset

# ========= I/O & TIME =========

OUT      = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/Wind_5km_10mi_meter.nc"

DT_SEC   = 600                    # 10-min forcing
N_DAYS   = 3                      # duration
START_TIME_STR = "2011-04-01 00:00:00"
CALENDAR = "gregorian"

# ========= PHYSICS (IDL_HURR_* equivalents) =========
rho_a = 1.2
p_n   = 1.012e5
p_c   = 9.68e4
Rmax  = 5.0e4                 # m
rad_edge    = 10.0
rad_ambient = 15.0
Umax  = 50.0                  # m/s
transpeed   = 5.0             # m/s
transdir    = np.deg2rad(180.0)
X0 = 3.432e6                  # m
Y0 = 9.0e5                    # m
f  = 5.5659e-5                # s^-1
t0_sec = 0

# Inflow angle (Zhang & Uhlhorn 2012)
A0_0 = -14.33; A0_Rnorm = -0.9;  A0_speed = -0.09
A1_0 =  0.14;  A1_Rnorm =  0.04; A1_speed =  0.05
P1_0 = 85.31;  P1_Rnorm =  6.88; P1_speed = -9.60
Deg2Rad = np.pi/180.0

def Cd_from_du10(du10):
    # Sullivan-style piecewise Cd(du10)
    return np.where(du10 < 11.0, 1.2e-3,
           np.where(du10 <= 20.0, (0.49 + 0.065*du10)*1e-3, 1.8e-3))

# ========= GRID (Cartesian domain + angular labels) =========
# physical size (m)
Lx_m = 3.0e6      # 3000 km
Ly_m = 1.8e6      # 1800 km

# model tracer resolution: 5 km → 5000 m
DX_tracer = 5.0e3
DY_tracer = 5.0e3

Nx = int(Lx_m / DX_tracer)   # 600
Ny = int(Ly_m / DY_tracer)   # 360

# 1-D physical coordinates (m) – cell centers
x_m_1d = (np.arange(Nx) + 0.5) * DX_tracer
y_m_1d = (np.arange(Ny) + 0.5) * DY_tracer

# 2-D physical plane (m) for stress_slice
X_m, Y_m = np.meshgrid(x_m_1d, y_m_1d, indexing="xy")

# simple_cartesian_grid angular coordinates (degrees)
lon_min, lon_max = -13.5, 13.5   # 27° span
lat_min, lat_max = -8.1, 8.1     # 16.2° span

dlam = (lon_max - lon_min) / Nx
dphi = (lat_max - lat_min) / Ny

Lon_1d_deg = lon_min + (np.arange(Nx) + 0.5) * dlam
Lat_1d_deg = lat_min + (np.arange(Ny) + 0.5) * dphi

# ---------- TIME (add +DT_SEC tail to avoid bracketing error) ----------
t_sec = np.arange(0, N_DAYS*86400 + DT_SEC, DT_SEC, dtype="int64")  # includes tail
t_min = (t_sec // 60).astype("int32")
Nt    = t_sec.size

# ========= PRECOMPUTE CONSTANTS =========
dp   = p_n - p_c
U_TS = 0.5*transpeed*np.cos(transdir)
V_TS = 0.5*transpeed*np.sin(transdir)
# Holland B (dimensionally consistent form)
B    = (Umax**2) * rho_a * np.e / dp

# U10 at edge for taper (scalar)
rrB_e  = rad_edge**(-B)
tmpA_e = (rrB_e*B) * dp
r_e    = rad_edge * Rmax
tmpB_e = (0.5*r_e*f) * rho_a
num_e  = tmpA_e * np.exp(-rrB_e)
den_e  = tmpB_e + np.sqrt((tmpA_e*rho_a)*np.exp(-rrB_e) + tmpB_e**2)
U10_edge = float(num_e / max(den_e, 1e-20))

# ========= ONE TIME-SLICE SOLVER =========
def stress_slice(tsec):
    XC = X0 + (tsec - t0_sec) * transpeed * np.cos(transdir)
    YC = Y0 + (tsec - t0_sec) * transpeed * np.sin(transdir)

    dX = X_m - XC
    dY = Y_m - YC
    r  = np.hypot(dX, dY)
    rr = r / Rmax

    # U10 magnitude
    U10 = np.zeros_like(r, dtype=np.float64)
    core = (rr > 1e-3) & (rr <= rad_edge)
    if np.any(core):
        rrB = np.power(rr[core], -B)
        tmpA = (rrB*B) * dp
        tmpB = (0.5*r[core]*f) * rho_a
        num  = tmpA * np.exp(-rrB)
        den  = tmpB + np.sqrt((tmpA*rho_a)*np.exp(-rrB) + tmpB**2)
        U10[core] = num / np.maximum(den, 1e-20)

    shell = (rr > rad_edge) & (rr < rad_ambient)
    if np.any(shell):
        fac = (rad_ambient - rr[shell])/(rad_ambient - rad_edge)
        U10[shell] = fac * U10_edge

    # Inflow angle
    Adir = np.arctan2(dY, dX)
    RSTR = np.minimum(rad_edge, rr)
    A0 = (A0_Rnorm*RSTR + A0_speed*Umax) + A0_0
    A1 = -A0*((A1_Rnorm*RSTR + A1_speed*transpeed) + A1_0)
    P1 = ((P1_Rnorm*RSTR + P1_speed*transpeed) + P1_0) * Deg2Rad
    Alph = (A0 - A1*np.cos((transdir - Adir) - P1)) * Deg2Rad
    if np.any(rr > rad_edge):
        taper = np.clip((rad_ambient - rr)/(rad_ambient - rad_edge), 0.0, 1.0)
        Alph = np.where(rr > rad_edge, Alph*taper, Alph)

    # 10 m winds (+0.5 * translation)
    u10x = U10*np.sin(Adir - np.pi - Alph) + U_TS
    u10y = U10*np.cos(Adir - Alph)        + V_TS
    du10 = np.hypot(u10x, u10y)

    # Stress (Pa)
    Cd   = Cd_from_du10(du10)
    Taux = (rho_a * Cd * du10 * u10x).astype(np.float32)
    Tauy = (rho_a * Cd * du10 * u10y).astype(np.float32)

    # Cast winds to float32 for storage
    u10x = u10x.astype(np.float32)
    u10y = u10y.astype(np.float32)

    return Taux, Tauy, u10x, u10y

# ========= WRITE NETCDF (NETCDF4_CLASSIC) =========
os.makedirs(os.path.dirname(OUT), exist_ok=True)
nc = Dataset(OUT, "w", format="NETCDF4_CLASSIC")

# Globals
nc.title       = "Hurricane wind & wind stress (A-grid east/north), 10-min cadence (+tail)"
nc.institution = "GFDL/NOAA (generated)"
nc.source      = "Idealized hurricane wind; Lon/Lat in degrees (simple_cartesian_grid)"
nc.history     = "Created by wind_writer_meters_coords.py"
nc.Conventions = "CF-1.7"

# Dims
nc.createDimension("Time", None)
nc.createDimension("Lat", Ny)
nc.createDimension("Lon", Nx)

# Time
vT = nc.createVariable("Time", "i4", ("Time",))
vT.units = f"minutes since {START_TIME_STR}"
vT.calendar = CALENDAR
vT.long_name = "time"
vT.standard_name = "time"
vT.axis = "T"; vT.cartesian_axis = "T"
vT[:] = t_min  # includes the tail record

# 1-D coordinate variables (degrees)
vLat = nc.createVariable("Lat", "f8", ("Lat",))
vLon = nc.createVariable("Lon", "f8", ("Lon",))

vLat.units = "degrees_north"
vLon.units = "degrees_east"
vLat.standard_name = "latitude"
vLon.standard_name = "longitude"
vLat.long_name = "latitude"
vLon.long_name = "longitude"
vLat.axis = "Y"; vLon.axis = "X"
vLat.cartesian_axis = "Y"; vLon.cartesian_axis = "X"

vLat[:] = Lat_1d_deg
vLon[:] = Lon_1d_deg

# Fields
FILL = np.float32(9.96921e36)
csy  = min(256, Ny); csx = min(256, Nx)

vtx = nc.createVariable("Taux", "f4", ("Time","Lat","Lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vty = nc.createVariable("Tauy", "f4", ("Time","Lat","Lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vu10 = nc.createVariable("u10", "f4", ("Time","Lat","Lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)
vv10 = nc.createVariable("v10", "f4", ("Time","Lat","Lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)

vtx.units = "Pa"; vty.units = "Pa"
vtx.standard_name = "surface_downward_eastward_stress"
vty.standard_name = "surface_downward_northward_stress"
vtx.long_name = "surface wind stress, eastward"
vty.long_name = "surface wind stress, northward"

vu10.units = "m s-1"; vv10.units = "m s-1"
vu10.standard_name = "eastward_wind"
vv10.standard_name = "northward_wind"
vu10.long_name = "10-m eastward wind"
vv10.long_name = "10-m northward wind"

# point to coord vars
for v in (vtx, vty, vu10, vv10):
    v.coordinates = "Lat Lon"
    v.missing_value = FILL

# Stream out
for k, tsec in enumerate(t_sec):
    tx2d, ty2d, u10x, u10y = stress_slice(int(tsec))
    vtx[k, :, :] = tx2d
    vty[k, :, :] = ty2d
    vu10[k, :, :] = u10x
    vv10[k, :, :] = u10y

nc.close()

# cadence hint for data_table
records_per_day = int(round(86400/DT_SEC))
print(f"[OK] wrote {OUT}")
print(f"[INFO] Nt={Nt} (includes +1 tail); use cadence = -{records_per_day} in data_table.")


In [ ]:
#!/usr/bin/env python3
# Hurricane wind + wind-stress for MOM6 data_table ("bilinear")  2m/s input file generate
# - Output: Taux/Tauy/u10/v10(Time, Lat, Lon) with 1-D Lat/Lon in degrees
# - Time axis includes a +1 record tail to avoid "after range" errors

import os
import numpy as np
from netCDF4 import Dataset

# ========= I/O & TIME =========

OUT      = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/Wind_5km_10mi_meter_2mps.nc"

DT_SEC   = 600                    # 10-min forcing
N_DAYS   = 3                      # duration
START_TIME_STR = "2011-04-01 00:00:00"
CALENDAR = "gregorian"

# ========= PHYSICS (IDL_HURR_* equivalents) =========
rho_a = 1.2
p_n   = 1.012e5
p_c   = 9.68e4
Rmax  = 5.0e4                 # m
rad_edge    = 10.0
rad_ambient = 15.0
Umax  = 50.0                  # m/s
transpeed   = 2.0             # m/s
transdir    = np.deg2rad(180.0)
X0 = 3.1728e6                  # m
Y0 = 9.0e5                    # m
f  = 5.5659e-5                # s^-1
t0_sec = 0

# Inflow angle (Zhang & Uhlhorn 2012)
A0_0 = -14.33; A0_Rnorm = -0.9;  A0_speed = -0.09
A1_0 =  0.14;  A1_Rnorm =  0.04; A1_speed =  0.05
P1_0 = 85.31;  P1_Rnorm =  6.88; P1_speed = -9.60
Deg2Rad = np.pi/180.0

def Cd_from_du10(du10):
    # Sullivan-style piecewise Cd(du10)
    return np.where(du10 < 11.0, 1.2e-3,
           np.where(du10 <= 20.0, (0.49 + 0.065*du10)*1e-3, 1.8e-3))

# ========= GRID (Cartesian domain + angular labels) =========
# physical size (m)
Lx_m = 3.0e6      # 3000 km
Ly_m = 1.8e6      # 1800 km

# model tracer resolution: 5 km → 5000 m
DX_tracer = 5.0e3
DY_tracer = 5.0e3

Nx = int(Lx_m / DX_tracer)   # 600
Ny = int(Ly_m / DY_tracer)   # 360

# 1-D physical coordinates (m) – cell centers
x_m_1d = (np.arange(Nx) + 0.5) * DX_tracer
y_m_1d = (np.arange(Ny) + 0.5) * DY_tracer

# 2-D physical plane (m) for stress_slice
X_m, Y_m = np.meshgrid(x_m_1d, y_m_1d, indexing="xy")

# simple_cartesian_grid angular coordinates (degrees)
lon_min, lon_max = -13.5, 13.5   # 27° span
lat_min, lat_max = -8.1, 8.1     # 16.2° span

dlam = (lon_max - lon_min) / Nx
dphi = (lat_max - lat_min) / Ny

Lon_1d_deg = lon_min + (np.arange(Nx) + 0.5) * dlam
Lat_1d_deg = lat_min + (np.arange(Ny) + 0.5) * dphi

# ---------- TIME (add +DT_SEC tail to avoid bracketing error) ----------
t_sec = np.arange(0, N_DAYS*86400 + DT_SEC, DT_SEC, dtype="int64")  # includes tail
t_min = (t_sec // 60).astype("int32")
Nt    = t_sec.size

# ========= PRECOMPUTE CONSTANTS =========
dp   = p_n - p_c
U_TS = 0.5*transpeed*np.cos(transdir)
V_TS = 0.5*transpeed*np.sin(transdir)
# Holland B (dimensionally consistent form)
B    = (Umax**2) * rho_a * np.e / dp

# U10 at edge for taper (scalar)
rrB_e  = rad_edge**(-B)
tmpA_e = (rrB_e*B) * dp
r_e    = rad_edge * Rmax
tmpB_e = (0.5*r_e*f) * rho_a
num_e  = tmpA_e * np.exp(-rrB_e)
den_e  = tmpB_e + np.sqrt((tmpA_e*rho_a)*np.exp(-rrB_e) + tmpB_e**2)
U10_edge = float(num_e / max(den_e, 1e-20))

# ========= ONE TIME-SLICE SOLVER =========
def stress_slice(tsec):
    XC = X0 + (tsec - t0_sec) * transpeed * np.cos(transdir)
    YC = Y0 + (tsec - t0_sec) * transpeed * np.sin(transdir)

    dX = X_m - XC
    dY = Y_m - YC
    r  = np.hypot(dX, dY)
    rr = r / Rmax

    # U10 magnitude
    U10 = np.zeros_like(r, dtype=np.float64)
    core = (rr > 1e-3) & (rr <= rad_edge)
    if np.any(core):
        rrB = np.power(rr[core], -B)
        tmpA = (rrB*B) * dp
        tmpB = (0.5*r[core]*f) * rho_a
        num  = tmpA * np.exp(-rrB)
        den  = tmpB + np.sqrt((tmpA*rho_a)*np.exp(-rrB) + tmpB**2)
        U10[core] = num / np.maximum(den, 1e-20)

    shell = (rr > rad_edge) & (rr < rad_ambient)
    if np.any(shell):
        fac = (rad_ambient - rr[shell])/(rad_ambient - rad_edge)
        U10[shell] = fac * U10_edge

    # Inflow angle
    Adir = np.arctan2(dY, dX)
    RSTR = np.minimum(rad_edge, rr)
    A0 = (A0_Rnorm*RSTR + A0_speed*Umax) + A0_0
    A1 = -A0*((A1_Rnorm*RSTR + A1_speed*transpeed) + A1_0)
    P1 = ((P1_Rnorm*RSTR + P1_speed*transpeed) + P1_0) * Deg2Rad
    Alph = (A0 - A1*np.cos((transdir - Adir) - P1)) * Deg2Rad
    if np.any(rr > rad_edge):
        taper = np.clip((rad_ambient - rr)/(rad_ambient - rad_edge), 0.0, 1.0)
        Alph = np.where(rr > rad_edge, Alph*taper, Alph)

    # 10 m winds (+0.5 * translation)
    u10x = U10*np.sin(Adir - np.pi - Alph) + U_TS
    u10y = U10*np.cos(Adir - Alph)        + V_TS
    du10 = np.hypot(u10x, u10y)

    # Stress (Pa)
    Cd   = Cd_from_du10(du10)
    Taux = (rho_a * Cd * du10 * u10x).astype(np.float32)
    Tauy = (rho_a * Cd * du10 * u10y).astype(np.float32)

    # Cast winds to float32 for storage
    u10x = u10x.astype(np.float32)
    u10y = u10y.astype(np.float32)

    return Taux, Tauy, u10x, u10y

# ========= WRITE NETCDF (NETCDF4_CLASSIC) =========
os.makedirs(os.path.dirname(OUT), exist_ok=True)
nc = Dataset(OUT, "w", format="NETCDF4_CLASSIC")

# Globals
nc.title       = "Hurricane wind & wind stress (A-grid east/north), 10-min cadence (+tail)"
nc.institution = "GFDL/NOAA (generated)"
nc.source      = "Idealized hurricane wind; Lon/Lat in degrees (simple_cartesian_grid)"
nc.history     = "Created by wind_writer_meters_coords.py"
nc.Conventions = "CF-1.7"

# Dims
nc.createDimension("Time", None)
nc.createDimension("Lat", Ny)
nc.createDimension("Lon", Nx)

# Time
vT = nc.createVariable("Time", "i4", ("Time",))
vT.units = f"minutes since {START_TIME_STR}"
vT.calendar = CALENDAR
vT.long_name = "time"
vT.standard_name = "time"
vT.axis = "T"; vT.cartesian_axis = "T"
vT[:] = t_min  # includes the tail record

# 1-D coordinate variables (degrees)
vLat = nc.createVariable("Lat", "f8", ("Lat",))
vLon = nc.createVariable("Lon", "f8", ("Lon",))

vLat.units = "degrees_north"
vLon.units = "degrees_east"
vLat.standard_name = "latitude"
vLon.standard_name = "longitude"
vLat.long_name = "latitude"
vLon.long_name = "longitude"
vLat.axis = "Y"; vLon.axis = "X"
vLat.cartesian_axis = "Y"; vLon.cartesian_axis = "X"

vLat[:] = Lat_1d_deg
vLon[:] = Lon_1d_deg

# Fields
FILL = np.float32(9.96921e36)
csy  = min(256, Ny); csx = min(256, Nx)

vtx = nc.createVariable("Taux", "f4", ("Time","Lat","Lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vty = nc.createVariable("Tauy", "f4", ("Time","Lat","Lon"),
                        zlib=True, complevel=2,
                        chunksizes=(1, csy, csx),
                        fill_value=FILL)
vu10 = nc.createVariable("u10", "f4", ("Time","Lat","Lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)
vv10 = nc.createVariable("v10", "f4", ("Time","Lat","Lon"),
                         zlib=True, complevel=2,
                         chunksizes=(1, csy, csx),
                         fill_value=FILL)

vtx.units = "Pa"; vty.units = "Pa"
vtx.standard_name = "surface_downward_eastward_stress"
vty.standard_name = "surface_downward_northward_stress"
vtx.long_name = "surface wind stress, eastward"
vty.long_name = "surface wind stress, northward"

vu10.units = "m s-1"; vv10.units = "m s-1"
vu10.standard_name = "eastward_wind"
vv10.standard_name = "northward_wind"
vu10.long_name = "10-m eastward wind"
vv10.long_name = "10-m northward wind"

# point to coord vars
for v in (vtx, vty, vu10, vv10):
    v.coordinates = "Lat Lon"
    v.missing_value = FILL

# Stream out
for k, tsec in enumerate(t_sec):
    tx2d, ty2d, u10x, u10y = stress_slice(int(tsec))
    vtx[k, :, :] = tx2d
    vty[k, :, :] = ty2d
    vu10[k, :, :] = u10x
    vv10[k, :, :] = u10y

nc.close()

# cadence hint for data_table
records_per_day = int(round(86400/DT_SEC))
print(f"[OK] wrote {OUT}")
print(f"[INFO] Nt={Nt} (includes +1 tail); use cadence = -{records_per_day} in data_table.")


In [ ]:
##2mps VS 5mps forcing comparing
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# =============================
# 1) files
# =============================
PATH_5 = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/Wind_5km_10mi_meter.nc"
PATH_2 = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/Wind_5km_10mi_meter_2mps.nc"

# domain size (km) consistent with your forcing generator
DOMAIN_X_KM = 3000.0
DOMAIN_Y_KM = 1800.0

# =============================
# 2) storm params (must match each forcing generator)
# =============================
storm_5 = dict(
    transpeed=5.0, transdir_deg=180.0,
    X0_m=3.432e6, Y0_m=9.0e5, t0_sec=0.0
)

storm_2 = dict(
    transpeed=2.0, transdir_deg=180.0,
    X0_m=3.1728e6, Y0_m=9.0e5, t0_sec=0.0
)

# =============================
# 3) choose the SAME storm-relative location (km)
# =============================
# dx_rel: along-track (positive east), dy_rel: cross-track (positive north)
# For a westward-moving storm, "right-hand side" is to the NORTH (dy_rel > 0).
DX_REL_KM = 0.0
DY_REL_KM = 100.0   # e.g., 100 km north of center (you can try 50, 100, 200)

# analysis: treat first 24h as adjustment; plot in storm-relative time
T_REL0_DAY = 1.0  # day when storm center reaches x=3000 km (by design)

# =============================
# helpers
# =============================
def time_to_days(time_da):
    t = time_da.values
    # your Time is i4 minutes since..., so numeric branch will be used
    if np.issubdtype(time_da.dtype, np.integer) or np.issubdtype(time_da.dtype, np.floating):
        units = (time_da.attrs.get("units","") or "").lower()
        if "minutes since" in units:
            return t.astype(float) / 1440.0
        if "hours since" in units:
            return t.astype(float) / 24.0
        if "seconds since" in units:
            return t.astype(float) / 86400.0
        return t.astype(float) / 86400.0
    # fallback for datetime64
    t0 = t[0]
    out = np.array([(ti - t0) / np.timedelta64(1, "D") for ti in t], dtype=float)
    return out

def nearest_index(coord_vals, target):
    return int(np.argmin(np.abs(coord_vals - target)))

def km_to_deg_linear(coord_1d, target_km, domain_km):
    """Map 0..domain_km linearly to coord min..max (degrees grid)."""
    cmin = float(coord_1d.min())
    cmax = float(coord_1d.max())
    frac = np.clip(target_km / domain_km, 0.0, 1.0)
    return cmin + frac * (cmax - cmin)

def sample_storm_relative_series(ds, storm, dx_rel_km, dy_rel_km):
    # identify coordinate names
    if "Lon" in ds.coords and "Lat" in ds.coords:
        xname, yname = "Lon", "Lat"
        xdim,  ydim  = "Lon", "Lat"
        x_units = (ds[xname].attrs.get("units","") or "").lower()
        y_units = (ds[yname].attrs.get("units","") or "").lower()
        is_deg = ("degree" in x_units) and ("degree" in y_units)
    elif "xh" in ds.coords and "yh" in ds.coords:
        xname, yname = "xh", "yh"
        xdim,  ydim  = "xh", "yh"
        is_deg = False
    else:
        raise RuntimeError(f"Unknown horizontal coords: {list(ds.coords)}")

    # time
    t_days = time_to_days(ds["Time"])
    t_sec  = t_days * 86400.0

    # storm center in km
    U = storm["transpeed"]
    th = np.deg2rad(storm["transdir_deg"])
    X0_km = storm["X0_m"] / 1000.0
    Y0_km = storm["Y0_m"] / 1000.0
    t0    = storm["t0_sec"]

    Xc_km = X0_km + (t_sec - t0) * U * np.cos(th) / 1000.0
    Yc_km = Y0_km + (t_sec - t0) * U * np.sin(th) / 1000.0

    # target point (storm-relative)
    Xt_km = Xc_km + dx_rel_km
    Yt_km = Yc_km + dy_rel_km

    xcoord = ds[xname].values
    ycoord = ds[yname].values

    taux = np.full_like(t_days, np.nan, dtype=float)
    tauy = np.full_like(t_days, np.nan, dtype=float)

    # quick sanity check: where is center at t=1 day?
    i1 = np.argmin(np.abs(t_days - 1.0))
    print(f"[CHECK] t≈1 day: Xc={Xc_km[i1]:.1f} km, Yc={Yc_km[i1]:.1f} km (expect Xc≈3000, Yc≈900)")

    # loop over time (Nt ~ 433, ok)
    for k in range(len(t_days)):
        # skip if target outside physical domain
        if (Xt_km[k] < 0) or (Xt_km[k] > DOMAIN_X_KM) or (Yt_km[k] < 0) or (Yt_km[k] > DOMAIN_Y_KM):
            continue

        if is_deg:
            xt = km_to_deg_linear(ds[xname], Xt_km[k], DOMAIN_X_KM)
            yt = km_to_deg_linear(ds[yname], Yt_km[k], DOMAIN_Y_KM)
        else:
            xt, yt = Xt_km[k], Yt_km[k]

        ix = nearest_index(xcoord, xt)
        jy = nearest_index(ycoord, yt)

        taux[k] = float(ds["Taux"].isel(Time=k, **{xdim: ix, ydim: jy}).values)
        tauy[k] = float(ds["Tauy"].isel(Time=k, **{xdim: ix, ydim: jy}).values)

    return t_days, taux, tauy

# =============================
# run
# =============================
ds5 = xr.open_dataset(PATH_5)
ds2 = xr.open_dataset(PATH_2)

t5, tx5, ty5 = sample_storm_relative_series(ds5, storm_5, DX_REL_KM, DY_REL_KM)
t2, tx2, ty2 = sample_storm_relative_series(ds2, storm_2, DX_REL_KM, DY_REL_KM)

# storm-relative time (days since entry at day=1)
tr5 = t5 - T_REL0_DAY
tr2 = t2 - T_REL0_DAY

fig, axes = plt.subplots(1, 2, figsize=(11, 3.6), constrained_layout=True, sharex=True)
ax1, ax2 = axes

label_loc = f"(dx={DX_REL_KM:.0f} km, dy={DY_REL_KM:.0f} km)"

ax1.plot(tr5, tx5, label=f"5 m/s {label_loc}")
ax1.plot(tr2, tx2, label=f"2 m/s {label_loc}")
ax1.axhline(0.0)
ax1.set_xlabel("days since entry (t - 1 day)")
ax1.set_ylabel("Surface τx (Pa)")
ax1.set_title("τx at same storm-relative location")
ax1.legend()

ax2.plot(tr5, ty5, label=f"5 m/s {label_loc}")
ax2.plot(tr2, ty2, label=f"2 m/s {label_loc}")
ax2.axhline(0.0)
ax2.set_xlabel("days since entry (t - 1 day)")
ax2.set_ylabel("Surface τy (Pa)")
ax2.set_title("τy at same storm-relative location")
ax2.legend()

plt.show()


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
###ocean model and wave model comparing data. The Wind_5km_10mi_meter.nc is the one obtained by 
#above code; ww3.20110401.nc is the file obtained after running two-way coupling using this wind-forcing
FORCING_FILE = "/archive/Qian.Xiao/Qian.Xiao/MOM_GLS_LT/3D_hurricane/Wind_5km_10mi_meter.nc"
WW3_FILE     = "/archive/Qian.Xiao/Qian.Xiao/FMS_Wave_Coupling_GOTM_kapp/3D_hurricane/3D_kappa_ePBL_waveLT_5km/5_results/ww3.20110401.nc"
# pick a physical target (km) to compare point time series (optional)
X_TARGET_KM = 2500.0
Y_TARGET_KM =  900.0
DOMAIN_X_KM = 3000.0
DOMAIN_Y_KM = 1800.0

def as_datetime64_from_num(time_vals, units, calendar="standard"):
    """netCDF numeric time -> numpy datetime64[ns]"""
    dt = num2date(time_vals, units=units, calendar=calendar)
    # num2date returns cftime or datetime objects; convert robustly
    return np.array([np.datetime64(str(d)) for d in dt], dtype="datetime64[ns]")

def stats(arr):
    arr = np.asarray(arr)
    m   = np.nanmean(arr)
    rms = np.sqrt(np.nanmean(arr**2))
    mx  = np.nanmax(np.abs(arr))
    return m, rms, mx

# -----------------------
# 1) Read WW3 uwnd/vwnd
# -----------------------
ww3 = Dataset(WW3_FILE)

# Confirm names in your file if needed
# print(ww3.variables.keys())

uw = ww3.variables["uwnd"][:]   # expect (time, lat, lon) or (time, y, x)
vw = ww3.variables["vwnd"][:]

tW_num = ww3.variables["time"][:]
tW_units = getattr(ww3.variables["time"], "units", None)
tW_cal   = getattr(ww3.variables["time"], "calendar", "standard")
if tW_units is None:
    raise RuntimeError("WW3 time variable has no 'units' attribute; cannot decode time safely.")

tW = as_datetime64_from_num(tW_num, tW_units, tW_cal)

lon2d = ww3.variables["longitude"][:]  # often (lat, lon)
lat2d = ww3.variables["latitude"][:]

# Your pattern:
lon1d = lon2d[0, :]
lat1d = lat2d[:, 0]

# Build DataArrays
da_uw = xr.DataArray(
    uw, dims=["time", "Lat", "Lon"],
    coords={"time": tW, "Lat": lat1d, "Lon": lon1d},
    name="uwnd"
)
da_vw = xr.DataArray(
    vw, dims=["time", "Lat", "Lon"],
    coords={"time": tW, "Lat": lat1d, "Lon": lon1d},
    name="vwnd"
)

ww3.close()

# -----------------------
# 2) Read forcing u10/v10
# -----------------------
dsF = xr.open_dataset(FORCING_FILE, decode_times=True)

# forcing uses coords Lon/Lat and var Time (your generator sets Time as a variable)
# xarray usually exposes it as dsF["Time"] already decoded
u10 = dsF["u10"]
v10 = dsF["v10"]

# Rename forcing time dim to "time" so we can interp cleanly
# (often u10 dims are ("Time","Lat","Lon") exactly)
u10 = u10.rename({"Time": "time"})
v10 = v10.rename({"Time": "time"})

# Ensure time coord is datetime64
# (decode_times=True may give cftime; convert to datetime64[ns])
tF_raw = u10["time"].values
if not np.issubdtype(np.array(tF_raw).dtype, np.datetime64):
    tF = np.array([np.datetime64(str(d)) for d in tF_raw], dtype="datetime64[ns]")
    u10 = u10.assign_coords(time=tF)
    v10 = v10.assign_coords(time=tF)

# -----------------------
# 3) Interp forcing -> WW3 grid + WW3 times
# -----------------------
# spatial interp (bilinear on rectilinear coords)
u10_onW = u10.interp(Lat=da_uw["Lat"], Lon=da_uw["Lon"])
v10_onW = v10.interp(Lat=da_uw["Lat"], Lon=da_uw["Lon"])

# time interp to WW3 output times
u10_onW = u10_onW.interp(time=da_uw["time"])
v10_onW = v10_onW.interp(time=da_uw["time"])

# -----------------------
# 4) Compare
# -----------------------
du = (da_uw - u10_onW)
dv = (da_vw - v10_onW)

mu, rmsu, maxu = stats(du.values)
mv, rmsv, maxv = stats(dv.values)

print("[COMPARE] WW3 (uwnd/vwnd) - Forcing (u10/v10) after interp")
print(f"  du: mean={mu:.3e}  rms={rmsu:.3e}  maxabs={maxu:.3e}  (m/s)")
print(f"  dv: mean={mv:.3e}  rms={rmsv:.3e}  maxabs={maxv:.3e}  (m/s)")

# -----------------------
# 5) Point time series at (X_TARGET_KM, Y_TARGET_KM)
#     map km->(Lon,Lat) using fractional position across domain
# -----------------------
frac_x = np.clip(X_TARGET_KM / DOMAIN_X_KM, 0.0, 1.0)
frac_y = np.clip(Y_TARGET_KM / DOMAIN_Y_KM, 0.0, 1.0)

lon_target = float(da_uw["Lon"].min()) + frac_x * float(da_uw["Lon"].max() - da_uw["Lon"].min())
lat_target = float(da_uw["Lat"].min()) + frac_y * float(da_uw["Lat"].max() - da_uw["Lat"].min())

ix = int(np.argmin(np.abs(da_uw["Lon"].values - lon_target)))
iy = int(np.argmin(np.abs(da_uw["Lat"].values - lat_target)))

print(f"[POINT] target ~({X_TARGET_KM:.0f} km, {Y_TARGET_KM:.0f} km) "
      f"-> nearest WW3 (Lon,Lat)=({da_uw['Lon'].values[ix]:.3f}, {da_uw['Lat'].values[iy]:.3f})")

tplot = da_uw["time"].values
uW_pt = da_uw.isel(Lat=iy, Lon=ix).values
vW_pt = da_vw.isel(Lat=iy, Lon=ix).values
uF_pt = u10_onW.isel(Lat=iy, Lon=ix).values
vF_pt = v10_onW.isel(Lat=iy, Lon=ix).values

fig, ax = plt.subplots(2, 1, figsize=(10, 6), constrained_layout=True, sharex=True)
ax[0].plot(tplot, uF_pt, label="forcing u10 (->WW3 grid/time)")
ax[0].plot(tplot, uW_pt, "--", label="WW3 uwnd")
ax[0].set_ylabel("u (m/s)"); ax[0].legend()

ax[1].plot(tplot, vF_pt, label="forcing v10 (->WW3 grid/time)")
ax[1].plot(tplot, vW_pt, "--", label="WW3 vwnd")
ax[1].set_ylabel("v (m/s)"); ax[1].legend()
plt.show()

# -----------------------
# 6) Storm “track proxy”: location of max wind speed vs time
# -----------------------
spdW = np.hypot(da_uw, da_vw)
spdF = np.hypot(u10_onW, v10_onW)

def maxloc(spd):
    lon_out = []
    lat_out = []
    for k in range(spd.sizes["time"]):
        sl = spd.isel(time=k)
        idx = np.nanargmax(sl.values)
        j, i = np.unravel_index(idx, sl.shape)
        lat_out.append(float(sl["Lat"].values[j]))
        lon_out.append(float(sl["Lon"].values[i]))
    return np.array(lon_out), np.array(lat_out)

lonW, latW = maxloc(spdW)
lonF, latF = maxloc(spdF)

fig, ax = plt.subplots(1, 1, figsize=(7, 5), constrained_layout=True)
ax.plot(lonF, latF, label="forcing max|U| track")
ax.plot(lonW, latW, "--", label="WW3 max|U| track")
ax.set_xlabel("Lon (deg)"); ax.set_ylabel("Lat (deg)")
ax.legend()
plt.show()
